<a href="https://colab.research.google.com/github/unverciftci/agents_intro/blob/main/Notebook_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI with Memory
### We want to build an agent that remembers conversations and learns facts


Model, chat function and tools as before

In [ ]:
import re
from datetime import datetime
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load AI (same as previous tutorials)
print("Loading AI...")
model_name = "Qwen/Qwen3-0.6B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
print("✅ AI ready!")

# Step 1: Tools from Tutorial 2
def calculator(expression):
    """Calculator tool"""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except:
        return "Error: Invalid calculation"

def get_weather(city):
    """Weather tool"""
    weather_data = {
        "paris": "Sunny, 22°C",
        "london": "Rainy, 15°C",
        "tokyo": "Cloudy, 18°C",
        "new york": "Windy, 20°C"
    }
    return weather_data.get(city.lower(), "Weather data not available")

def remember_fact(fact):
    """Tool to remember facts"""
    try:
        if " is " in fact:
            key, value = fact.split(" is ", 1)
            memory.learn_fact(key.strip(), value.strip())
            return f"I'll remember that {key.strip()} is {value.strip()}"
        else:
            memory.learn_fact("note", fact)
            return f"I'll remember: {fact}"
    except:
        return "I couldn't understand what to remember"

TOOLS = {
    "calculator": calculator,
    "get_weather": get_weather,
    "remember": remember_fact
}

print("✅ Tools ready: calculator, weather, remember")

# Step 2: AI function
def ask_ai(prompt):
    """Send prompt to AI"""
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True, enable_thinking=False
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=130,
            temperature=0.3,
            do_sample=True,
        )

    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    return tokenizer.decode(output_ids, skip_special_tokens=True).strip()

Let's create our agent's memory system


In [ ]:
# Step 3: Create memory system
class AgentMemory:
    """Simple memory system for the agent"""

    def __init__(self):
        self.conversation_history = []
        self.learned_facts = {}

    def add_conversation(self, user_message, agent_response):
        """Remember this conversation"""
        self.conversation_history.append({
            'timestamp': datetime.now().strftime("%H:%M"),
            'user': user_message,
            'agent': agent_response
        })

        # Keep only last 8 conversations to avoid token limits
        if len(self.conversation_history) > 8:
            self.conversation_history = self.conversation_history[-8:]

    def learn_fact(self, key, value):
        """Learn a new fact"""
        self.learned_facts[key] = value
        print(f"🧠 Learned: {key} = {value}")

    def get_context(self):
        """Build context from memory"""
        context = ""

        # Add learned facts
        if self.learned_facts:
            context += "What I know about you:\n"
            for key, value in self.learned_facts.items():
                context += f"- {key}: {value}\n"
            context += "\n"

        # Add recent conversations
        if self.conversation_history:
            context += "Recent conversation:\n"
            for conv in self.conversation_history[-3:]:  # Last 3 only
                context += f"[{conv['timestamp']}] You: {conv['user']}\n"
                context += f"[{conv['timestamp']}] Me: {conv['agent']}\n"
            context += "\n"

        return context

# Create memory instance
memory = AgentMemory()
print("✅ Memory system ready")

This is our smart agent with memory

In [ ]:
# Step 4: Smart agent with memory
def smart_agent(user_message):
    """Agent with memory and tools"""

    # Get memory context
    context = memory.get_context()

    # System prompt with memory and tools
    system_prompt = f"""You are a helpful AI assistant with memory and tools.

{context}

Available tools:
- TOOL:calculator:expression (for math)
- TOOL:get_weather:city (for weather)
- TOOL:remember:fact (to remember something about the user)

Examples:
User: What's 25 + 17?
AI: I'll calculate that. TOOL:calculator:25+17

User: My name is Sarah
AI: Nice to meet you! TOOL:remember:name is Sarah

User: What's the weather in Tokyo?
AI: Let me check that for you. TOOL:get_weather:tokyo

Current message: {user_message}
Response:"""

    print(f"👤 User: {user_message}")

    # Get AI response
    ai_response = ask_ai(system_prompt)
    print(f"🤖 AI: {ai_response}")

    # Handle tool calls
    if "TOOL:" in ai_response:
        tool_match = re.search(r'TOOL:(\w+):(.+)', ai_response)
        if tool_match:
            tool_name = tool_match.group(1)
            tool_input = tool_match.group(2).strip()

            print(f"🛠️ Using tool: {tool_name}({tool_input})")

            if tool_name in TOOLS:
                tool_result = TOOLS[tool_name](tool_input)
                print(f"✅ Tool result: {tool_result}")

                # Get final response
                final_prompt = system_prompt + ai_response + f"\nTool result: {tool_result}\nFinal response:"
                final_response = ask_ai(final_prompt)

                # Remember this conversation
                memory.add_conversation(user_message, final_response)

                return f"{ai_response}\nTool result: {tool_result}\n{final_response}"

    # Remember normal conversations too
    memory.add_conversation(user_message, ai_response)
    return ai_response

Test memory

In [ ]:
# Step 5: Test memory
print("\n🧪 Testing memory:")

test_conversation = [
    "Hi! My name is Alex",
    "I'm 25 years old",
    "I live in Paris",
    "What's my name?",
    "How old am I?",
    "What's the weather where I live?"
]

for message in test_conversation:
    print(f"\n--- Message ---")
    response = smart_agent(message)
    print(f"Final response: {response}")
    print("-" * 30)

# Step 6: Show memory contents
print("\n🧠 What's in memory:")
print("\n📚 Learned Facts:")
for key, value in memory.learned_facts.items():
    print(f"  • {key}: {value}")

print("\n💬 Recent Conversations:")
for conv in memory.conversation_history[-3:]:
    print(f"  [{conv['timestamp']}] User: {conv['user']}")
    print(f"  [{conv['timestamp']}] Agent: {conv['agent']}")

# Step 7: Interactive mode
def chat_with_memory():
    """Interactive chat with memory-enabled agent"""
    print("\n🧠 Chat with your smart agent!")
    print("Tell it about yourself and test its memory!")
    print("Type 'memory' to see what it remembers")
    print("Type 'quit' to exit")

    while True:
        user_input = input("\n👤 You: ")

        if user_input.lower() == 'quit':
            print("👋 Goodbye!")
            break
        elif user_input.lower() == 'memory':
            print("\n🧠 Memory contents:")
            print("Facts:", memory.learned_facts)
            print("Conversations:", len(memory.conversation_history))
            continue

        if user_input.strip():
            response = smart_agent(user_input)
            print(f"🤖 Agent: {response}")

# Ready to test memory? Uncomment the line below:
# chat_with_memory()

print("\n🎉 Your agent now has memory!")
print("🚀 Next: Tutorial 4 - Take real actions")